In [15]:
from CARP_solver import *
from cProfile import run

In [16]:
info = read_file("../datasets/egl-s1-A.dat")
gv.init(info)

In [17]:
def hamming_distance(s1: Solution, s2: Solution):
    def deadhead(solu: Solution):
        deadhead_link = []
        for route in solu.routes:
            for idx in range(len(route.tasks) - 1):
                deadhead_link.append((route[idx].t, route[idx + 1].s))

        return deadhead_link

    m = min(len(s1.routes), len(s2.routes))
    n = gv.num_tasks
    common = len(set(deadhead(s1)).intersection(set(deadhead(s2))))
    return n + m - common


def pop_dist_rank(pop):
    """计算每个solu在pop中的平均距离，倒序排序，越大排名越高"""
    pop_size = len(pop)  # ! 最后要减一
    dist_matrix = np.zeros((pop_size, pop_size), dtype=int)
    for i in range(pop_size):
        for j in range(i + 1, pop_size):
            dist = hamming_distance(pop[i], pop[j])
            dist_matrix[i, j] = dist_matrix[j, i] = dist
    
    # 添加随机值[0,1)进行排序
    return np.argsort(-dist_matrix.sum(axis=0))


def QDNS_update_pop(pop, solu):
    if not solu.feasible():
        return False

    pop_size = len(pop)
    pop.append(solu)

    cost_rank = np.argsort(np.array([solu.cost for solu in pop]) + np.random.rand(pop_size + 1))
    dist_rank = pop_dist_rank(pop)
    alpha = 0.6
    QDF = [alpha * cost_rank[i] + (1 - alpha) * dist_rank[i] for i in range(pop_size + 1)]
    # 将要被抛弃的pop的idx
    worst = np.argmax(QDF)
    # print(list(map(lambda x: x.cost, pop)))
    # print(QDF, worst)
    pop.pop(worst)  # 直接在原来的pop上改
    if worst == pop_size:
        # 弹出了新加的solu
        return False

    return True
    
    

In [18]:
def main(pop_size, timeout):
    pop = init_pop(pop_size)
    best_solu = min(pop, key=lambda x: x.eval())
    best_eval = best_solu.eval()
    tabu_list = []
    max_len_tabu = 30

    start = end = perf_counter()
    while end - start < timeout:
        # crossover
        co = sample(pop, k=2)
        new_solu = Solution.crossover(co[0], co[1])

        # 生成一个随机数，判断应该进行怎样improve
        ty = randint(0, 2)
        if ty == 0:
            pass
        elif ty == 1:
            # single insertion
            new_solu = single_insert(new_solu, best_eval, tabu_list, 10)
            pass

        
        elif ty == 2:
            # merge split
            new_solu = merge_split(new_solu)
        
        if new_solu is not None and new_solu not in tabu_list:
            pop = update_pop(pop, new_solu)
            # new_solu.assert_demand()
        
            tabu_list.append(new_solu)

        bfs = best_feasible_solu(pop)
        eval = bfs.eval()
        if eval < best_eval:
            best_eval = eval
            print(eval)

        # 删除禁忌表中留存时间过长的解
        if len(tabu_list) >= max_len_tabu:
            tabu_list.pop(0)

        end = perf_counter()
    
    return best_feasible_solu(pop)

In [19]:
# run("print(main(20, 5))")
best = main(20, 100)

14062
12494
12363
11800
9214
8049
7952
7926
7841
7722
7574
7566
7270
7267
7243
7231
7167
7108
7046
6948
6851
6594
6490
6445
6440
6331
6291
6274
6238
6222
6215
6212
6177
6075
5996
5992
5972
5915
5745
5736
5704
5696
5686
5606
5547
5539
5487
5458


In [20]:
best

solution cost:5458, feasible
{'tasks': [(0, 0): 0, (86, 85): 4, (85, 84): 20, (84, 83): 10, (83, 81): 13, (81, 79): 16, (35, 37): 25, (39, 38): 18, (38, 37): 6, (35, 36): 20, (36, 42): 16, (45, 76): 16, (76, 77): 15, (77, 78): 12, (78, 79): 12, (0, 0): 0], 'cost': 642, 'remain_cap': 7}
{'tasks': [(0, 0): 0, (111, 109): 13, (109, 110): 7, (109, 106): 8, (106, 105): 28, (108, 107): 103, (107, 106): 25, (106, 111): 13, (112, 113): 12, (0, 0): 0], 'cost': 505, 'remain_cap': 1}
{'tasks': [(0, 0): 0, (104, 105): 15, (94, 95): 8, (95, 96): 17, (97, 96): 3, (65, 66): 33, (66, 68): 4, (68, 70): 6, (71, 70): 9, (71, 72): 18, (72, 43): 36, (44, 33): 4, (44, 43): 14, (43, 42): 31, (42, 45): 5, (0, 0): 0], 'cost': 828, 'remain_cap': 7}
{'tasks': [(0, 0): 0, (115, 116): 19, (116, 118): 24, (123, 125): 11, (129, 125): 35, (117, 113): 58, (115, 0): 18, (0, 0): 0], 'cost': 419, 'remain_cap': 45}
{'tasks': [(0, 0): 0, (65, 61): 37, (61, 62): 7, (62, 63): 9, (54, 53): 10, (8, 7): 37, (5, 4): 26, (5, 7): 

In [ ]:
# QDNS_update_pop(pop, pop[0])